# Reinforcement learning (RL)

## 1. 相关概念

#### 智能体 (agent):

强化学习的主体，做动作或决策。

#### 环境 (environment):

与智能体交互的对象。

#### 状态 (state):

对当前时刻环境的概括。

#### 状态空间 (state space):

所有可能存在的状态的集合，记作$\mathcal{S}$。可离散可连续，可有限可无限。

#### 动作 (action):

智能体根据当前状态所做的决策。

#### 动作空间 (action space):

所有可能的动作的集合，记作$\mathcal{A}$。可离散可连续，可有限可无限。

#### 奖励 (reward):

智能体执行一个动作后，环境返回给智能体的一个数值。

#### 状态转移 (state transition):

智能体执行动作$a$，从$t$时刻的状态$s$转换到$t+1$时刻的状态$s'$。状态转移函数为: $p(s' \mid s, a) = \mathbb{P}(S_{t+1}=s' \mid S_t = s_t, A_t = a_t)$。

#### 策略 (policy):

如何根据观测到的状态做出对执行动作的决策，可随机可确定。

#### 智能体与环境交互 (agent environment interaction):

智能体观测环境的状态$s$，根据策略$\pi$执行动作$a$，环境反馈给智能体奖励$r$和新的状态$s'$。

#### 回合 (episode):

从开始到结束。

#### 随机性:

智能体执行动作时策略可能具有随机性，状态转移也可能具有随机性。

#### 马尔可夫性质 (Markov property):

下一时刻的状态$S_{t+1}$仅依赖于当前时刻的状态$S_t$和动作$A_t$，即$\mathbb{P}(S_{t+1} \mid S_t, A_t) = \mathbb{P}(S_{t+1} \mid S_1, A_1, \cdots, S_t, A_t)$。

#### 轨迹 (trajectory):

一个回合中，智能体观测到的所有状态、动作和奖励: $s_1, a_1, r_1, s_2, a_2, r_2, \cdots$。

#### 回报 (return) 或累计奖励 (cumulative future reward):

从当前时刻到回合结束所有奖励的总和: $U_t = R_t + \cdots + R_n$。

#### 最优策略 (optimum policy):

使得回报期望最大化的策略。

#### 折扣回报 (discounted return):

使用折扣率 (discount factor)给未来的奖励打折扣: $U_t = R_t + \gamma \cdot R_{t+1} + \gamma^2 \cdot R_{t+2} + \cdots$。

#### 有限期 (finite-horizon) 和无限期 (infinite-horizon):

马尔可夫过程是否存在终止状态 (terminal state)。对于无限期，需满足$\gamma < 1$，否则回报会无穷大。

## 2. 价值函数

#### 动作价值函数 (action-value function):

$Q_\pi(s_t, a_t) = \mathbb{E}_{S_{t+1}, A_{t+1}, \cdots, S_n, A_n}\left[ U_t \mid S_t = s_t, A_t = a_t \right]$，依赖于$s_t, a_t, \pi$，不依赖于$t + 1$时刻及以后的状态和动作。

#### 最优动作价值函数 (optimal action-value function):

$Q_*(s_t, a_t) = \max_\pi Q_\pi(s_t, a_t), \forall s_t \in S_t, a_t \in A_t$。消除了策略的影响，只评价当前状态和动作的好坏。

#### 状态价值函数 (state-value function):

$V_\pi(s_t) = \mathbb{E}_{a_t \sim \pi(\cdot \mid s_t)}[Q_\pi(s_t, a_t)]$用来衡量策略$\pi$与当前状态$s_t$的好坏。

# Deep Q Network (DQN)

## 1. 相关概念

#### DQN (deep Q network):

使用神经网络$Q(s, a; w)$近似最优动作价值函数$Q_*(s, a)$。

#### 最优贝尔曼方程 (optimal Bellman equation):

给定四元组$(s_t, a_t, r_t, s_{t+1})$，可计算出$r_t + \gamma \cdot \max_{a \in \mathcal{A}} Q_*(s_{t + 1}, a)$，可看作下式的蒙特卡洛近似$\mathbb{E}_{S_{t+1} \sim p(\cdot \mid s_t, a_t)}\left[ R_t + \gamma \cdot \max_{A \in \mathcal{A}} Q_*(S_{t+1}, A) \mid S_t = s_t, A_t = a_t \right]$.

#### 时间差分 (temporal difference, TD):

$U_t = \sum_{k=t}^n \gamma^{k-t} \cdot R_k, U_{t+1} = \sum_{k=t+1}^n \gamma^{k-t-1} \cdot R_k$，从而$U_t = R_t + \gamma \cdot U_{t+1}, Q_*(s_t, a_t) \approx r_t + \gamma \cdot \max_{a \in \mathcal{A}} Q_*(s_{t+1}, a)$。使用神经网络: $Q(s_t, a_t; w) \approx r_t + \gamma \cdot \max_{a \in \mathcal{A}} Q(s_{t+1}, a; w)$，左边称为预测$\hat{q}_t$，右边称为TD目标$\hat{y}_t$。

#### TD算法:

TD目标$\hat{y}_t$相较于预测$\hat{q}_t$，部分基于事实，因此更可信。损失函数: $L(w) = 1/2 \cdot \left[Q(s_t, a_t; w) - \hat{y}_t \right]^2$，梯度$\Delta_w L(w) = (\hat{q}_t - \hat{y}_t) \cdot \Delta_w Q(s_t, a_t; w)$，则网络参数更新公式$w \leftarrow w - \alpha \cdot \delta_t \cdot \Delta_w Q(s_t, a_t; w)$，其中$\delta_t = (\hat{q}_t - \hat{y}_t)$称为TD误差。

#### 行为策略 (behavior policy) 和目标策略 (target policy):

行为策略控制智能体与环境交互，目的是收集经验。目标策略是强化学习得到的目标函数。

#### 同策略 (on-policy) 和异策略 (off-policy):

行为策略和目标策略相同为同策略，否则为异策略。

#### $\epsilon$-greedy:

一种行为策略，可以初始时$\epsilon$值较大，之后不断减小。

$$
a_t = 
\begin{cases} 
\text{argmax}_a \; Q(s_t, a; w) & \text{with probability } (1 - \epsilon) \\ 
\text{uniformly extract from } \mathcal{A} & \text{with probability } \epsilon 
\end{cases}
$$

#### 经验回放 (experience replay):

将智能体与环境交互的记录$(s_t, a_t, r_t, s_{t+1})$储存在经验回放缓存 (experience replay buffer) 中。经验回放打破了序列的相关性，并重复利用收集到的经验。经验回放通过行为策略收集的，不用于正在训练的目标策略，所以经验回放适用于异策略，而不适用于同策略。

## 2. Double DQN (DDQN)

### 2.1. 自举 (bootstrapping) 导致偏差传播

如果$Q(s_{t+1}, a_{t+1}; w)$是对真实价值$Q_*(s_{t+1}, a_{t+1})$的高估或低估，就会导致$Q(s_{t}, a_{t}; w)$高估或低估$Q_*(s_{t}, a_{t})$，高估或低估不断传播，从$(s_{t+1}, a_{t+1})$到$(s_{t}, a_{t})$。

### 2.2. 最大化导致高估

若DQN的输出是真实价值$Q_*(s, a)$加上均值为0的噪声$\epsilon$: $Q(s, a; w) = Q_*(s, a) + \epsilon$，则$\mathbb{E}_\epsilon \left[ \max_{a \in \mathcal{A}} Q(s, a; w) \right] \geq \max_{a \in \mathcal{A}} Q_*(s, a)$，从而TD目标$\hat{y}_t$通常是对真实价值的高估。

### 2.3. 高估的危害

均匀的高估不会对算法表现造成显著影响，但是非均匀高估可能会导致算法性能下降。

### 2.4. DDQN原理

TD目标$\hat{y}_t = r_t + \gamma \cdot \max_{a \in \mathcal{A}} Q(s_{t+1}, a; w)$，分为两步: (1) 选择: $a^* = \text{argmax}_{a \in \mathcal{A}} Q(s_{t+1}, a; w)$; (2) 求值: $\hat{y}_t = r_t + Q(s_{t+1}, a^*; w)$。使用DQN做选择，使用目标网络 (target network) 做求值$\hat{y}_t = r_t + Q(s_{t+1}, a^*; w^-)$。

### 2.5. 训练流程

设DQN和目标网络的当前参数为$w_{now}$和$w_{now}^-$。

(1) 前向传播，得到$\hat{q}_t = Q(s_t, a_t; w_{now}), a^* = \text{argmax}_{a \in \mathcal{A}} Q(s_{t+1}, a; w_{now})$和$\hat{q}_{t+1} = Q(s_{t+1}, a^*; w_{now}^-)$;

(2) 计算TD目标和TD误差: $\hat{y}_t = r_t + \gamma \cdot \hat{q}_{t+1}, \delta_t = \hat{q}_t - \hat{y}_t$;

(3) 反向传播: $g_t = \Delta_w Q(s_t, a_t; w_{now})$;

(4) 参数更新: $w_{new} = w_{now} - \alpha \cdot \delta_t \cdot g_t$和$w_{new}^- = \tau \cdot w_{new} + (1 - \tau) \cdot w_{now}^-$.

### 2.6. warm up：

(1) 由于Q网络尚未训练，warm up阶段智能体随机探索，可减少初期的偏差。

(2) 更广泛地探索状态-动作空间，提高探索性。

(3) 收集初始经验放入经验缓存，减少由于数据不足导致的过拟合。

# 代码实现

In [1]:
# 导入库
import os
import gym
import argparse
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [2]:
# 定义Q-Network
class QNet(nn.Module):
    def __init__(self, state_size, num_actions):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_actions)
    
    def forward(self, state):
        x = F.relu(self.fc1(state.float()))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# 定义DQN类
class DQN:
    def __init__(self, state_size, num_actions, gamma, device):
        self.gamma = gamma
        self.Q = QNet(state_size, num_actions).to(device)
        self.target_Q = QNet(state_size, num_actions).to(device)
        self.target_Q.load_state_dict(self.Q.state_dict())
        
    def get_action(self, state):
        # state: (state_size,)
        # qvals: (num_actions,)
        qvals = self.Q(state)
        return qvals.argmax().item()
    
    def compute_loss(self, bs, ba, br, bd, bns):
        # bs, bns: (batch_size, state_size)
        # ba, br, bd: (batch_size,)
        # qvals, next_qvals: (batch_size,)
        qvals = self.Q(bs).gather(1, ba.unsqueeze(1)).squeeze(1)
        # next_actions: (batch_size, 1)
        next_actions = self.Q(bns).argmax(dim=1, keepdim=True)
        # detach() 避免对目标网络的参数求导
        next_qvals = self.target_Q(bns).gather(1, next_actions).squeeze(1).detach()
        loss = F.mse_loss(qvals, br + self.gamma * next_qvals * (1 - bd))
        return loss        
    
    def soft_update(self, tau=0.01):
        for target_param, param in zip(self.target_Q.parameters(), self.Q.parameters()):
            target_param.data.copy_(param.data * tau + target_param.data * (1 - tau))

In [4]:
# 定义经验缓存
class ReplayBuffer:
    def __init__(self, max_size, device):
        self.max_size = max_size
        self.device = device
        self.size = 0
        self.state = []
        self.action = []
        self.reward = []
        self.done = []
        self.next_state = []
        
    def push(self, state, action, reward, done, next_state):
        if self.size < self.max_size:
            self.state.append(state)
            self.action.append(action)
            self.reward.append(reward)
            self.done.append(done)
            self.next_state.append(next_state)
        else:
            idx = self.size % self.max_size
            self.state[idx] = state
            self.action[idx] = action
            self.reward[idx] = reward
            self.done[idx] = done
            self.next_state[idx] = next_state
        self.size += 1
        
    def sample(self, n):
        sample_num = min(self.size, self.max_size)
        indices = np.random.choice(range(sample_num), size=n, replace=True) if self.size < n else np.random.choice(range(sample_num), size=n, replace=False)
        state = torch.tensor([self.state[i] for i in indices], dtype=torch.float32, device=self.device)
        action = torch.tensor([self.action[i] for i in indices], dtype=torch.long, device=self.device)
        reward = torch.tensor([self.reward[i] for i in indices], dtype=torch.float32, device=self.device)
        done = torch.tensor([self.done[i] for i in indices], dtype=torch.float32, device=self.device)
        next_state = torch.tensor([self.next_state[i] for i in indices], dtype=torch.float32, device=self.device)
        return state, action, reward, done, next_state

In [5]:
# 训练
def train(args, env):
    agent = DQN(args.state_size, args.num_actions, args.discount, args.device)
    replay_buffer = ReplayBuffer(10000, args.device)
    optimizer = torch.optim.Adam(agent.Q.parameters(), lr=args.lr)
    writer = SummaryWriter()
    
    epsilon = 1
    epsilon_max = 1
    epsilon_min = 0.1
    
    episode_reward = 0
    episode_length = 0
    episode_num = 1
    max_episode_reward = -float('inf')
    
    agent.Q.train()
    state, _ = env.reset()
    
    for i in range(args.max_steps):
        if np.random.rand() < epsilon or i < args.warmup_steps:
            action = env.action_space.sample()
        else:
            action = agent.get_action(torch.from_numpy(state).to(args.device))
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward
        episode_length += 1
        replay_buffer.push(state, action, reward, done, next_state)
        state = next_state
        
        if done:
            if episode_reward > max_episode_reward:
                max_episode_reward = episode_reward
                save_path = os.path.join(args.output_dir, 'model.bin')
                torch.save(agent.Q.state_dict(), save_path)
                
            writer.add_scalar('Maximum reward', max_episode_reward, episode_num)
            writer.add_scalar('Episode reward', episode_reward, episode_num)
            writer.add_scalar('Episode length', episode_length, episode_num)
            print(f'step = {i}, reward = {episode_reward:.0f}, length = {episode_length}, max reward = {max_episode_reward}, epsilon = {epsilon:.3f}')
                     
            episode_reward = 0
            episode_length = 0
            episode_num += 1
            epsilon = max(epsilon - (epsilon_max - epsilon_min) * args.epsilon_decay, epsilon_min)
            state, _ = env.reset()
            
        if i > args.warmup_steps:
            bs, ba, br, bd, bns = replay_buffer.sample(n=args.batch_size)
            loss = agent.compute_loss(bs, ba, br, bd, bns)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            agent.soft_update()
            if i % 500 == 0:
                writer.add_scalar('Loss', loss.item(), i)
                
    writer.close()

In [6]:
# 测试
def eval(args, env):
    agent = DQN(args.state_size, args.num_actions, args.discount, args.device)
    model_path = os.path.join(args.output_dir, 'model.bin')
    agent.Q.load_state_dict(torch.load(model_path))
    agent.Q.to(args.device)
    agent.Q.eval()
    
    episode_reward = 0
    episode_length = 0
    state, _ = env.reset()
    for _ in range(5000):
        action = agent.get_action(torch.from_numpy(state).to(args.device))
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward
        episode_length += 1
        state = next_state
        
        if done:
            state, _ = env.reset()
            print(f'episode reward = {episode_reward:.0f}, episode length = {episode_length}')
            episode_reward = 0
            episode_length = 0

In [7]:
# 运行
args = argparse.Namespace()
args.env = 'CartPole-v1'
args.state_size = 4
args.num_actions = 2
args.discount = 0.99
args.max_steps = int(1e5)
args.lr = 1e-3
args.batch_size = 32
args.warmup_steps = int(1e4)
args.output_dir = 'output'
args.epsilon_decay = 1 / 1000
if torch.backends.mps.is_available():
    args.device = torch.device('mps')
elif torch.cuda.is_available():
    args.device = torch.device('cuda')
else:
    args.device = torch.device('cpu')
    
os.makedirs(args.output_dir, exist_ok=True)
    
env = gym.make(args.env)
env.reset(seed=42)
env.action_space.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if args.device == torch.device('cuda'):
    torch.cuda.manual_seed(42)

print('Training started...')
train(args, env)
print('Training completed!')

print('Evaluation started...')
eval(args, env)
print('Evaluation completed!')

Training started...
step = 28, reward = 29, length = 29, max reward = 29.0, epsilon = 1.000
step = 45, reward = 17, length = 17, max reward = 29.0, epsilon = 0.999
step = 114, reward = 69, length = 69, max reward = 69.0, epsilon = 0.998
step = 129, reward = 15, length = 15, max reward = 69.0, epsilon = 0.997
step = 168, reward = 39, length = 39, max reward = 69.0, epsilon = 0.996
step = 193, reward = 25, length = 25, max reward = 69.0, epsilon = 0.995
step = 207, reward = 14, length = 14, max reward = 69.0, epsilon = 0.995
step = 230, reward = 23, length = 23, max reward = 69.0, epsilon = 0.994
step = 258, reward = 28, length = 28, max reward = 69.0, epsilon = 0.993
step = 300, reward = 42, length = 42, max reward = 69.0, epsilon = 0.992
step = 325, reward = 25, length = 25, max reward = 69.0, epsilon = 0.991
step = 353, reward = 28, length = 28, max reward = 69.0, epsilon = 0.990
step = 370, reward = 17, length = 17, max reward = 69.0, epsilon = 0.989
step = 403, reward = 33, length =

step = 9831, reward = 27, length = 27, max reward = 73.0, epsilon = 0.591
step = 9866, reward = 35, length = 35, max reward = 73.0, epsilon = 0.590
step = 9892, reward = 26, length = 26, max reward = 73.0, epsilon = 0.590
step = 9903, reward = 11, length = 11, max reward = 73.0, epsilon = 0.589
step = 9941, reward = 38, length = 38, max reward = 73.0, epsilon = 0.588
step = 9984, reward = 43, length = 43, max reward = 73.0, epsilon = 0.587


/Users/xavier/PythonCode/venv/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)


step = 10018, reward = 34, length = 34, max reward = 73.0, epsilon = 0.586
step = 10037, reward = 19, length = 19, max reward = 73.0, epsilon = 0.585
step = 10052, reward = 15, length = 15, max reward = 73.0, epsilon = 0.584
step = 10061, reward = 9, length = 9, max reward = 73.0, epsilon = 0.583
step = 10079, reward = 18, length = 18, max reward = 73.0, epsilon = 0.582
step = 10093, reward = 14, length = 14, max reward = 73.0, epsilon = 0.581
step = 10105, reward = 12, length = 12, max reward = 73.0, epsilon = 0.581
step = 10128, reward = 23, length = 23, max reward = 73.0, epsilon = 0.580
step = 10145, reward = 17, length = 17, max reward = 73.0, epsilon = 0.579
step = 10154, reward = 9, length = 9, max reward = 73.0, epsilon = 0.578
step = 10167, reward = 13, length = 13, max reward = 73.0, epsilon = 0.577
step = 10178, reward = 11, length = 11, max reward = 73.0, epsilon = 0.576
step = 10190, reward = 12, length = 12, max reward = 73.0, epsilon = 0.575
step = 10199, reward = 9, len

step = 18315, reward = 297, length = 297, max reward = 300.0, epsilon = 0.489
step = 18338, reward = 23, length = 23, max reward = 300.0, epsilon = 0.488
step = 18460, reward = 122, length = 122, max reward = 300.0, epsilon = 0.487
step = 18600, reward = 140, length = 140, max reward = 300.0, epsilon = 0.486
step = 18640, reward = 40, length = 40, max reward = 300.0, epsilon = 0.485
step = 18761, reward = 121, length = 121, max reward = 300.0, epsilon = 0.484
step = 19019, reward = 258, length = 258, max reward = 300.0, epsilon = 0.483
step = 19213, reward = 194, length = 194, max reward = 300.0, epsilon = 0.482
step = 19267, reward = 54, length = 54, max reward = 300.0, epsilon = 0.482
step = 19343, reward = 76, length = 76, max reward = 300.0, epsilon = 0.481
step = 19385, reward = 42, length = 42, max reward = 300.0, epsilon = 0.480
step = 19430, reward = 45, length = 45, max reward = 300.0, epsilon = 0.479
step = 19685, reward = 255, length = 255, max reward = 300.0, epsilon = 0.47

step = 33372, reward = 365, length = 365, max reward = 493.0, epsilon = 0.392
step = 33608, reward = 236, length = 236, max reward = 493.0, epsilon = 0.392
step = 33888, reward = 280, length = 280, max reward = 493.0, epsilon = 0.391
step = 34261, reward = 373, length = 373, max reward = 493.0, epsilon = 0.390
step = 34369, reward = 108, length = 108, max reward = 493.0, epsilon = 0.389
step = 34434, reward = 65, length = 65, max reward = 493.0, epsilon = 0.388
step = 34764, reward = 330, length = 330, max reward = 493.0, epsilon = 0.387
step = 35031, reward = 267, length = 267, max reward = 493.0, epsilon = 0.386
step = 35212, reward = 181, length = 181, max reward = 493.0, epsilon = 0.385
step = 35380, reward = 168, length = 168, max reward = 493.0, epsilon = 0.384
step = 35620, reward = 240, length = 240, max reward = 493.0, epsilon = 0.383
step = 35963, reward = 343, length = 343, max reward = 493.0, epsilon = 0.383
step = 35984, reward = 21, length = 21, max reward = 493.0, epsilo

step = 53838, reward = 240, length = 240, max reward = 500.0, epsilon = 0.297
step = 53944, reward = 106, length = 106, max reward = 500.0, epsilon = 0.296
step = 54032, reward = 88, length = 88, max reward = 500.0, epsilon = 0.295
step = 54231, reward = 199, length = 199, max reward = 500.0, epsilon = 0.294
step = 54709, reward = 478, length = 478, max reward = 500.0, epsilon = 0.293
step = 54752, reward = 43, length = 43, max reward = 500.0, epsilon = 0.293
step = 55252, reward = 500, length = 500, max reward = 500.0, epsilon = 0.292
step = 55427, reward = 175, length = 175, max reward = 500.0, epsilon = 0.291
step = 55494, reward = 67, length = 67, max reward = 500.0, epsilon = 0.290
step = 55544, reward = 50, length = 50, max reward = 500.0, epsilon = 0.289
step = 55814, reward = 270, length = 270, max reward = 500.0, epsilon = 0.288
step = 55908, reward = 94, length = 94, max reward = 500.0, epsilon = 0.287
step = 56140, reward = 232, length = 232, max reward = 500.0, epsilon = 0.

step = 71095, reward = 124, length = 124, max reward = 500.0, epsilon = 0.201
step = 71124, reward = 29, length = 29, max reward = 500.0, epsilon = 0.200
step = 71248, reward = 124, length = 124, max reward = 500.0, epsilon = 0.199
step = 71574, reward = 326, length = 326, max reward = 500.0, epsilon = 0.198
step = 71604, reward = 30, length = 30, max reward = 500.0, epsilon = 0.197
step = 71743, reward = 139, length = 139, max reward = 500.0, epsilon = 0.196
step = 71887, reward = 144, length = 144, max reward = 500.0, epsilon = 0.195
step = 71909, reward = 22, length = 22, max reward = 500.0, epsilon = 0.194
step = 71929, reward = 20, length = 20, max reward = 500.0, epsilon = 0.194
step = 72039, reward = 110, length = 110, max reward = 500.0, epsilon = 0.193
step = 72163, reward = 124, length = 124, max reward = 500.0, epsilon = 0.192
step = 72308, reward = 145, length = 145, max reward = 500.0, epsilon = 0.191
step = 72386, reward = 78, length = 78, max reward = 500.0, epsilon = 0.

step = 93691, reward = 500, length = 500, max reward = 500.0, epsilon = 0.105
step = 93920, reward = 229, length = 229, max reward = 500.0, epsilon = 0.104
step = 94420, reward = 500, length = 500, max reward = 500.0, epsilon = 0.104
step = 94542, reward = 122, length = 122, max reward = 500.0, epsilon = 0.103
step = 94865, reward = 323, length = 323, max reward = 500.0, epsilon = 0.102
step = 95122, reward = 257, length = 257, max reward = 500.0, epsilon = 0.101
step = 95622, reward = 500, length = 500, max reward = 500.0, epsilon = 0.100
step = 96079, reward = 457, length = 457, max reward = 500.0, epsilon = 0.100
step = 96579, reward = 500, length = 500, max reward = 500.0, epsilon = 0.100
step = 97079, reward = 500, length = 500, max reward = 500.0, epsilon = 0.100
step = 97405, reward = 326, length = 326, max reward = 500.0, epsilon = 0.100
step = 97905, reward = 500, length = 500, max reward = 500.0, epsilon = 0.100
step = 98009, reward = 104, length = 104, max reward = 500.0, ep

#### Reward:

![fig1](https://raw.githubusercontent.com/Xavier-MaYiMing/Reinforcement-learning-and-combinatorial-optimzation/main/figs/DQN%20-%20reward.png)

#### Maximum reward:

![fig2](https://raw.githubusercontent.com/Xavier-MaYiMing/Reinforcement-learning-and-combinatorial-optimzation/main/figs/DQN%20-%20max%20reward.png)